# 1) Loading and cleaning the data

In [51]:
import pandas as pd
import numpy as np

In [90]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

#print('Before dropping Boroughs ='.join(df.shape))
df= df[df['Borough'] != 'Not assigned'].reset_index(drop= True)

#there are no Postal codes repeated so we dont need to concatinate the Neighbourhoods.

df.shape

(103, 3)

(103, 3)